In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/regression-5a-create-folds/train_raw_folds.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv')
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv')

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [ ]:
# only continous columns in df but filter for them specifically
cont_cols = [col for col in df if col.startswith('cont')]
df_test = df_test[cont_cols]

In [ ]:
final_predictions = []
for fold in range(5):
    x_train = df[df.kfold != fold].reset_index(drop=True)
    x_valid = df[df.kfold == fold].reset_index(drop=True)
    
    y_train = x_train.target
    y_valid = x_valid.target
    
    x_train = x_train[cont_cols]
    x_valid = x_valid[cont_cols]

    scaler = StandardScaler()
    
    x_train_transformed = scaler.fit_transform(x_train)
    x_valid_transformed = scaler.transform(x_valid)
    df_test_transformed = scaler.transform(df_test)
    
    
    model = XGBRegressor(random_state=4, n_jobs=4)
    model.fit(x_train, y_train)
    
    preds_valid = model.predict(x_valid_transformed)
    df_test_preds = model.predict(df_test_transformed)
    
    final_predictions.append(df_test_preds)
    print(fold, mean_squared_error(y_valid, preds_valid, squared=False))

In [ ]:
preds = np.mean(np.column_stack(final_predictions), axis=1)

In [ ]:
sample_submission.target = preds
sample_submission.to_csv('submission.csv', index=False)